In [1]:
#ultima sessione di lavoro (danilo)
#df.loc["esito"] = home_page_content, il codice viene eseguito correttamente, ma non si salva sul dataframa
#il problema è probabilmente l'inplace
#RISOLTO: bastava togliere loc


#Danilo:
#la funzione non dovrebbe avere un return?
#guardare siti rotti


In [2]:
#todo
#scrivere se il sito è offline (gestire response code diversi da 200)
#qualcosa da aggiustare sull'inserimento della lingua


In [3]:
import pandas as pd
from stop_words import get_stop_words
import re
from selenium import webdriver
import time
from langdetect import detect, detect_langs
from webdriver_manager.chrome import ChromeDriverManager
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor


In [4]:
#sistema per non dover installare il driver in ogni cartella
driver = webdriver.Chrome(ChromeDriverManager().install())


headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
'Accept-Encoding': 'none',
'Accept-Language': 'en-US,en;q=0.8',
'Connection': 'keep-alive'}

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/85.0.4183.87/chromedriver_win32.zip


[WDM] - Driver has been saved in cache [C:\Users\filip\.wdm\drivers\chromedriver\win32\85.0.4183.87]


In [5]:
#driver = webdriver.Chrome("webdriver/chromedriver")

In [6]:
#stopwords base importate

stop_words = get_stop_words('it')
stop_words += get_stop_words('en')
stop_words += get_stop_words('pt')
stop_words += get_stop_words('es')
stop_words += get_stop_words('de')
stop_words += get_stop_words('fr')

In [7]:
#stopwords personali aggiunte da me

stop_words += ["priority","target","events","eventname","flavors","callback","document","stato","legacy","window","visualizza"]
stop_words += ["module","article","cette","encore","faire","d’une"]
stop_words += ["c'est","della","delle"]
stop_words += ["settembre",	"positivi",	"giorno",	"giorni",	"agosto",	"settimana",	"progetto",	"weeks",	"evidenza",	"siniscola",	"della",	"lucia",	"santa"]

In [8]:
df = pd.read_excel("sites_to_check2.xlsx")

In [9]:
#sites_to_check_df
df# = df[:10]

,site,url_to_import,esito,lingua,parole_trovate
0,bimbieviaggi.it,http://bimbieviaggi.it,vuoto,NaN,NaN
1,altarimini.it,http://altarimini.it,vuoto,NaN,NaN
2,puretravel.com,http://puretravel.com,vuoto,NaN,NaN
3,reisebuch.de,http://reisebuch.de,vuoto,NaN,NaN
4,trivenetogoal.it,http://trivenetogoal.it,vuoto,NaN,NaN
...,...,...,...,...,...
123,ilgazzettinovesuviano.com,http://ilgazzettinovesuviano.com,vuoto,NaN,NaN
124,expartibus.it,http://expartibus.it,vuoto,NaN,NaN
125,targatocn.it,http://targatocn.it,vuoto,NaN,NaN
126,mrgreengold.it,http://mrgreengold.it,vuoto,NaN,NaN


In [10]:
#spostare su file separato

def stripPunctuations(home_page_content):
    
    #bad_chars = "?.,;:!\'"
    bad_chars = ["\n","\t","?",".",",",";",":","!","\\","’"]
    for char in home_page_content:
        if char in bad_chars:
            home_page_content = home_page_content.replace(char,"")
        
    return home_page_content
        


def avoidStopWords(wordlist, stop_words):
    for word in wordlist:
        if word not in stop_words:
            list_saved_words.append(word)
    return list_saved_words


def removeWordsWithBadChars(content):
    bad_list = ["/","(",")", "&", "{","’"]
    bad_list += ["}", "h1", "h2", "h3",]
    bad_list += ["px","_","\\","&","@", "#","-","%","'"]
    bad_list += ["1","2","3","4","5","6","7","8","9","0"]
    for bad_char in bad_list:
        for word in content:
            if bad_char in word:
                content.remove(word)
    return content


def CountFrequency(list_saved_words): 
    lista_2 = []
    
    for item in list_saved_words:
        local_list = [item, list_saved_words.count(item)]
        lista_2.append(local_list)
    
    return lista_2


def removeDuplicates(list_saved_words):
    unique_keywords = []
    for item in list_saved_words:
        if item not in unique_keywords:
            unique_keywords.append(item)
    return unique_keywords



def sortList(sub_li): 
  
    # key is set to sort using second element of  
    # sublist lambda has been used 
    sub_li.sort(reverse=True, key = lambda x: x[1]) 
    return sub_li 


In [11]:
def elaborate_web_page(url, df):
    home_page_content = ""
    temp_keyword_frequencies = [] #serve per eliminare delle stop-words extra
    frequency_dict = {}
    all_words = [] #serve per eliminare delle stop-words extra, superato 
    print(url)
    
    
    if "http://" not in url: #togliendo l'http dal df e poi dal file excel rendo visibile che l'url è già stato controllato
        print("già fatto")
        return
    else:
        driver = webdriver.Chrome("webdriver/chromedriver")
        home_page_content_saved = []
        index = df[df['url_to_import']==url].index.item()
        
        try:
            driver.get(url)
            print("url preso")
            time.sleep(5) #non sono sicuro che sia necessario, il mio timore è che certe pagine non carichino per intero (numero parole sempre diverso)
        except:
            print("non è un sito")
            df["esito"][index] = "error"
            df["url_to_import"][index] = df["url_to_import"][index].replace("http://","")
            return
            
        home_page_content = driver.find_element_by_tag_name('body').text
        home_page_content = str(home_page_content)
        print(home_page_content[:50])
        home_page_language = detect_langs(home_page_content)
        df["lingua"][index] = home_page_language
        print(home_page_language)

        home_page_content = home_page_content.lower()
        home_page_content = stripPunctuations(home_page_content)
        home_page_content = home_page_content.split(" ")
        print(len(home_page_content))
        #home_page_content diventa lista


        for word in home_page_content:
            if len(word)> 4 and len(word)< 20: 
                home_page_content_saved.append(word)
                home_page_content = home_page_content_saved

        #togliamo parole con bad_char
        home_page_content = removeWordsWithBadChars(home_page_content)
        #all_words += home_page_content

        #calcoliamo la frequenza, ma siamo sempre in una lista
        home_page_content = CountFrequency(home_page_content)

        #togliamo le keyword che sono state usate una volta sola
        temp_keyword_frequencies = []

        for item in home_page_content:
            if item[1] >= 2:
                temp_keyword_frequencies.append(item)
            home_page_content = temp_keyword_frequencies

        #sistemino per eliminare un po' di cripto-stopwords
        frequent_keywords = []

        for item in home_page_content:
            if item[1] >= 4:
                frequent_keywords.append(item)
        #print(frequent_keywords)


        #togliamo i duplicati
        home_page_content = removeDuplicates(home_page_content)
        #print(home_page_content)   

        #ordiniamo
        home_page_content = sortList(home_page_content)
        
        print(f"lunghezza finale:{len(home_page_content)}")
        df["parole_trovate"][index] = len(home_page_content)
        home_page_content = str(home_page_content) #prima di inserire i valori sul foglio excel trasformiamo in stringa

        df["esito"][index] = home_page_content
        df["url_to_import"][index] = df["url_to_import"][index].replace("http://","")
        driver.close()

        

    #finalmente si trasforma tutto in dizionario
    #il dizionario ha come key il sito, come value il mega-listone di keyword+frequency     
    #frequency_dict[url] = home_page_content



#si ricomincia da capo con un nuovo sito
    

    
#print(frequency_dict)

In [12]:
urls = df["url_to_import"].tolist()
print(urls)

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    future_to_url = {executor.submit(elaborate_web_page, url, df): url for url in urls}

['http://bimbieviaggi.it', 'http://altarimini.it', 'http://puretravel.com', 'http://reisebuch.de', 'http://trivenetogoal.it', 'http://louiseethelene.de', 'http://gudtechtricks.com', 'http://ilcaudino.it', 'http://travelfashiontips.com', 'http://siniscolanotizie.net', 'http://dottorsport.info', 'http://salentosport.net', 'http://techniblogic.com', 'http://hallo-frau.de', 'http://newsmondo.it', 'http://elclarinweb.com', 'http://giornaledipuglia.com', 'http://informapuglia.com', 'http://helpconsumatori.it', 'http://bioecogeo.com', 'http://romait.it', 'http://ilquotidianodellazio.it', 'http://travelnostop.com', 'http://sicurmoto.it', 'http://segundoenfoque.com', 'http://risparmioeinvestimento.it', 'http://polkadot.it', 'http://osprofanos.com', 'http://laflecha.net', 'http://otakupt.com', 'http://lifestyleblog.it', 'http://chedonna.it', 'http://dietagratis.com', 'http://psychologie-sante.tn', 'http://laprovinciadibiella.it', 'http://giornaledisondrio.it', 'http://bancario.info', 'http://ita

<ipython-input-11-cebea853ac66>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["lingua"][index] = home_page_language
C:\Users\filip\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-11-cebea853ac66>:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["parole_trovate"][index] = len(home_page_content)
<ipython-input-11-ce

[de:0.9999965240411558]
55
lunghezza finale:1
url preso
url preso
Verstanden!
Diese Webseite nutzt Cookies, um Ihnen
[de:0.9999976831803473]
72
lunghezza finale:3
url preso
GUD TECH TRICKS
Technology at Finger Tip
ANDROID
F
[en:0.9999962115824058]
521
lunghezza finale:40
http://dottorsport.info
Il rispetto della tua privacy è la nostra priorità
http://salentosport.net
[it:0.9999972289439583]
816
lunghezza finale:49
HOME
CONTATTI
CHI SIAMO
BLOG TOUR
EVENTI E FIERE
M
[it:0.9999952971727827]
1446
lunghezza finale:144
url preso
url preso
http://techniblogic.com
http://hallo-frau.de
Mercoledì 16 Settembre11:33:27
Segnala Notizia Acc
[it:0.9999972169325971]
679
lunghezza finale:19
url preso
Registro stampa
Lo Staff di Triveneto Goal
Informa
[it:0.7142832473188719, pt:0.14285902032251677, en:0.14285772772698485]
108
lunghezza finale:1
http://newsmondo.it
http://elclarinweb.com
url preso
Home
Holidays
Special Offers
Brochures
Specialists
[en:0.9999969138785432]
6479
lunghezza finale:50
url pre

lunghezza finale:58
lunghezza finale:75
http://chesuccede.it
http://studionews24.com
http://betnow365.it
http://4news.it
Home
About Us
Write For Us
Contact Us
ARCHITECTURE
[en:0.9999974147144554]
730
lunghezza finale:54
INDOOR
OUTDOOR
IMPROVEMENT
ABOUT
CONTACT
TAGS
BRIL
[en:0.9999964855537173]
506
lunghezza finale:25
http://teleuniverso.it
http://ferienhausmarkt.com
url preso
url preso
url preso
MODA
BELLEZZA
CASA
CUCINA
SALUTE MAMME E BIMBI
Boo
[it:0.9999973635905927]
549
lunghezza finale:29
4Comics
4Games
4Life
4Tech
4VR
Guide strategiche
R
[it:0.9999966271673669]
991
lunghezza finale:73
url preso
url preso
Area Riservata
Pronostici di calcio professionali 
[it:0.9999954662626578]
664
lunghezza finale:60
url preso
http://consigliando.it
url preso
http://aostasera.it
http://calcionow.it
http://fashiontimes.it
Questo sito utilizza cookie che ci aiutano a forni
[it:0.9999962261167052]
367
lunghezza finale:21
Il rispetto della tua privacy è la nostra priorità
[it:0.999996484633487]
1045


lunghezza finale:150
url preso
Skip to content
mercoledì, 16 Settembre 2020, 11:4
[it:0.9999942022512049]
48268
lunghezza finale:2857
non è un sito


<ipython-input-11-cebea853ac66>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["esito"][index] = "error"
<ipython-input-11-cebea853ac66>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["url_to_import"][index] = df["url_to_import"][index].replace("http://","")


In [13]:
#controllo se la funzione sta funzionando, poi si userà concurrent.futures.ThreadPoolExecutor

#urls = df["url_to_import"]


#for url in urls:
 #   elaborate_web_page(url, df)
    

In [18]:
df[:20]

,site,url_to_import,esito,lingua,parole_trovate
0,bimbieviaggi.it,bimbieviaggi.it,"[['bambini', 23], ['viaggio', 13], ['nostro', ...",[it:0.9999952971727827],144.0
1,altarimini.it,altarimini.it,"[['settembre', 25], ['rimini', 7], ['della', 7...",[it:0.9999972169325971],19.0
2,puretravel.com,puretravel.com,"[['islands', 10], ['operators', 6], ['local', ...",[en:0.9999969138785432],50.0
3,reisebuch.de,reisebuch.de,"[['jetzt', 4], ['anschauen', 4], ['bestellen',...",[de:0.9999976831803473],3.0
4,trivenetogoal.it,trivenetogoal.it,"[['qualsiasi', 2]]","[it:0.7142832473188719, pt:0.14285902032251677...",1.0
5,louiseethelene.de,louiseethelene.de,"[['herbst', 2]]",[de:0.9999965240411558],1.0
6,gudtechtricks.com,gudtechtricks.com,"[['…read', 10], ['development', 5], ['online',...",[en:0.9999962115824058],40.0
7,ilcaudino.it,ilcaudino.it,"[['giorni', 27], ['rassegna', 18], ['settembre...",[it:0.9999972289439583],49.0
8,travelfashiontips.com,travelfashiontips.com,"[['napoli', 6], ['viaggio', 6], ['quali', 4], ...",[it:0.9999964954717402],39.0
9,siniscolanotizie.net,siniscolanotizie.net,"[['della', 8], ['siniscola', 6], ['redazione',...",[it:0.9999965626419612],26.0


In [19]:
df.to_excel("sites_to_check2.xlsx",index=False)

In [16]:
#df.to_excel("sites_to_check_sbagliato.xlsx",index=False)

In [17]:
#eliminare stopwords extra
from pandas import DataFrame
#df_all_words = DataFrame(frequent_keywords)
df_all_words
df_all_words.to_excel("frequent_keywords.xlsx")

NameError: name 'df_all_words' is not defined